In [ ]:
import utils
import pandas as pd

In [ ]:
quizzes = utils.load_quizzes()

In [ ]:
answers = utils.load_log('answers')

In [ ]:
# Convert UTC timestamp to datetime
answers['timestamp'] = pd.to_datetime(answers['timestamp'], unit='ms')

# Only keep the latest complete answer for a given user/quiz pair
get_latest = lambda group: group.iloc[group.timestamp.argmax()]
did_complete_quiz = lambda row: len(row.answers) == len(quizzes[row.quizName]['questions'])
groups = ['sessionId', 'quizName']
answers = answers \
    .loc[lambda df: df.apply(did_complete_quiz, axis=1)] \
    .groupby(groups) \
    .apply(get_latest) \
    .drop(columns=groups) \
    .reset_index()
                                                    
answers['frac_correct'] = answers.answers.map(lambda a: len([o for o in a if o['correct']]) / len(a))

In [ ]:
answers.groupby('quizName').frac_correct.describe()[['count', 'mean', '50%']].sort_values('mean')